### MTH8211 - Lab 2

Par: Guillaume Thibault - 1948612

<br><br/>

Implementation de la factorisation $$ A = LDL* $$ d'une matric carrée hermitienne

* L : triangulaire inférieur unitaire (diagonale est composée que de 1)
* D est diagonale

In [23]:
using Distributions, LinearAlgebra
using SparseArrays

In [24]:
# Hermitian matrix: complex square matrix that is equal to its own conjugate transpose
A = Matrix(Hermitian(rand(3,3) + I))
@assert ishermitian(A)
display(A)

3×3 Matrix{Float64}:
 1.27564   0.276557  0.301389
 0.276557  1.54877   0.475295
 0.301389  0.475295  1.48424

#### Partie 1.
Si A est définie posotive, on peut garantir l'exsitence de la factorisation, cela est équivalent à la factorisation de Cholesky

1. Dans ce cas de figure, expliquer pourquoi 𝐷 sera également définie positive 

<br> <br/>

    Reponse


2. Algorithme pour a décomposition

    Avec 

    $$ A = \left[ {\begin{array}{cc}  a & b & c \\ b & * & * \\ c & * & * \end{array} } \right] $$

    Nous pouvons trouver la matrice suivante pour enlever l'élément de la première colonne

    $$ L_1 = \left[ {\begin{array}{cc}  1 & 0 & 0 \\ -b/a & 1 & 0 \\ -c/a & 0 & 1 \end{array} } \right] $$

    Comme ça, on peut utiliser $ L_1^T $ pour enlever les éléments de la première ligne.


    Avec $ A_1 = L_1 A L_2^T $, on applique la même procédure pour trouver $ A_2 = L_2 A_1 L_2^T = (L_2 L_1) A (L_2^T L_1^T) $. On continue ainsi jusqu'à la dernière ligne/colonne et on obtient $ D = A_{n-1} = (L_{n-1} ... L_1) A_0 (L_{n-1}^T ... L_1^T) $.

    Enfin, en fixant $ L = (L_{n-1}^T ... L_1^T)^{-1} $ on trouve la factorisation : $A=LDL^T$.

    

3. Read the documentation on $Symmetric$ and $Hermitian$

4. Implementation of the algo

In [26]:
function ldl(A)
    D = copy(A)
    L = triu(zeros(ComplexF32, size(A)) + I)
    m, n = size(A)
    for i in 1:m
        # Lₙ matrix for this it
        Lₙ = zeros(ComplexF32, size(A)) + I
        # Find the elements that will cancel the row
        for j in i+1:n
            # Populate Lₙ
            Lₙ[j, i] = -D[j, i] / D[i, i]
            # Fill L
            L[j, i] = -Lₙ[j, i]
        end
        # Find matrix for next iteration Lₙ D Lₙᵀ 
        D = Lₙ * D * Lₙ'
    end
    return L, D
end

ldl (generic function with 1 method)

Tests:

In [27]:
# Sample case
A = [1 2 2; 2 1 2; 2 2 1];
A = Symmetric(A); # triangle supérieur
display(A);

L, D = ldl(A);

display(L * D * L');

3×3 Symmetric{Int64, Matrix{Int64}}:
 1  2  2
 2  1  2
 2  2  1

3×3 Matrix{ComplexF32}:
 1.0+0.0im  2.0+0.0im  2.0+0.0im
 2.0+0.0im  1.0+0.0im  2.0+0.0im
 2.0+0.0im  2.0+0.0im  1.0+0.0im

In [28]:
# Using triangular sup matrix 
# A = Symmetric(tril(A' * A), :L); # triangle inférieur
A = rand(5, 5);
A = Symmetric(triu(A' * A));
display(A);
L, D = ldl(A);
display(L * D * L');


5×5 Symmetric{Float64, Matrix{Float64}}:
 2.15395  1.91469  1.19778   1.71641   1.03463
 1.91469  1.766    1.04323   1.57839   1.03206
 1.19778  1.04323  0.785903  0.863777  0.706922
 1.71641  1.57839  0.863777  1.63471   0.927483
 1.03463  1.03206  0.706922  0.927483  1.03176

5×5 Matrix{ComplexF64}:
 2.15395+0.0im  1.91469+0.0im   1.19778+0.0im   1.71641+0.0im   1.03463+0.0im
 1.91469+0.0im    1.766+0.0im   1.04323+0.0im   1.57839+0.0im   1.03206+0.0im
 1.19778+0.0im  1.04323+0.0im  0.785903+0.0im  0.863777+0.0im  0.706922+0.0im
 1.71641+0.0im  1.57839+0.0im  0.863777+0.0im   1.63471+0.0im  0.927483+0.0im
 1.03463+0.0im  1.03206+0.0im  0.706922+0.0im  0.927483+0.0im   1.03176+0.0im

In [29]:
A = rand(5, 5);
A = Symmetric(tril(A' * A), :L); # triangle inférieur
display(A);
L, D = ldl(A);
display(L * D * L');

5×5 Symmetric{Float64, Matrix{Float64}}:
 1.60656   1.5708    1.82164   0.890977  0.760883
 1.5708    1.96261   1.50156   0.785956  0.95811
 1.82164   1.50156   2.28994   1.00324   0.793074
 0.890977  0.785956  1.00324   0.755041  0.27338
 0.760883  0.95811   0.793074  0.27338   0.740055

5×5 Matrix{ComplexF64}:
  1.60656+0.0im    1.5708+0.0im  …  0.890977+0.0im  0.760883+0.0im
   1.5708+0.0im   1.96261+0.0im     0.785956+0.0im   0.95811+0.0im
  1.82164+0.0im   1.50156+0.0im      1.00324+0.0im  0.793074+0.0im
 0.890977+0.0im  0.785956+0.0im     0.755041+0.0im   0.27338+0.0im
 0.760883+0.0im   0.95811+0.0im      0.27338+0.0im  0.740055+0.0im

In [30]:
A = [1 0 2+2im 0 3-3im; 0 4 0 5 0; 6-6im 0 7 0 8+8im; 0 9 0 1 0; 2+2im 0 3-3im 0 4];
A = Hermitian(A);
display(A);

L, D = ldl(A);

display(L * D * L');

5×5 Hermitian{Complex{Int64}, Matrix{Complex{Int64}}}:
 1+0im  0+0im  2+2im  0+0im  3-3im
 0+0im  4+0im  0+0im  5+0im  0+0im
 2-2im  0+0im  7+0im  0+0im  8+8im
 0+0im  5+0im  0+0im  1+0im  0+0im
 3+3im  0+0im  8-8im  0+0im  4+0im

5×5 Matrix{ComplexF32}:
 1.0+0.0im  0.0+0.0im  2.0+2.0im  0.0+0.0im  3.0-3.0im
 0.0+0.0im  4.0+0.0im  0.0+0.0im  5.0+0.0im  0.0+0.0im
 2.0-2.0im  0.0+0.0im  7.0+0.0im  0.0+0.0im  8.0+8.0im
 0.0+0.0im  5.0+0.0im  0.0+0.0im  1.0+0.0im  0.0+0.0im
 3.0+3.0im  0.0+0.0im  8.0-8.0im  0.0+0.0im  4.0+0.0im

5. Function solve to solve `x = solve(L, D, b)`

In [31]:
function solve(L, D, b)
    # Solve Ly = b 
    y = L \ b
    # Then DL' x = y
    x = D * L' \ y
    
    return x
end

solve (generic function with 1 method)

Tests:

In [32]:
A = [1 2 2; 2 1 2; 2 2 1];
A = Symmetric(A);
b = [1; 2; 3]
x = A \ b
println(x)

L, D = ldl(A)

solve(L, D, b)

[1.4000000000000001, 0.4, -0.6]


3-element Vector{ComplexF32}:
        1.4f0 + 0.0f0im
 0.40000004f0 + 0.0f0im
       -0.6f0 - 0.0f0im

In [36]:
A = [1 0 2+2im 0 3-3im; 0 4 0 5 0; 6-6im 0 7 0 8+8im; 0 9 0 1 0; 2+2im 0 3-3im 0 4];
A = Hermitian(A);
b = [1, 2, 3, 4, 5]

x = A \ b
println(x)

L, D = ldl(A)

solve(L, D, b)

ComplexF64[0.508888888888889 - 0.21555555555555558im, 0.8571428571428571 - 0.0im, 0.20000000000000004 + 0.09777777777777785im, -0.2857142857142857 - 0.0im, 0.11111111111111109 - 0.015555555555555545im]


5-element Vector{ComplexF32}:
  0.5088889f0 - 0.21555579f0im
 0.85714287f0 + 0.0f0im
 0.20000005f0 + 0.09777784f0im
 -0.2857143f0 + 0.0f0im
 0.11111111f0 - 0.015555556f0im

## Part 2

Factorization on

 $$ A = \left[ {\begin{array}{cc}  M & A^* \\ A & -N \end{array} } \right] $$

Where 𝑀 and 𝑁 are positive definite Hermitian and 𝐴 is arbitrary (𝐴 can be rectangular).

Test your implementation on
quasi definite systems

In [43]:
# Create a positive definite Hermitian matrix
M = Hermitian(rand(Complex{Float64}, 6, 6));
N = -Hermitian(rand(Complex{Float64}, 4, 4));
A = rand(4, 6)

bloc_matrix = [M A'; A N]
display(bloc_matrix);


L, D = ldl(bloc_matrix)
display(L * D * L');


10×10 Matrix{ComplexF64}:
  0.455247+0.0im        0.190538+0.4716im    …    0.228847+0.0im
  0.190538-0.4716im     0.451102+0.0im            0.468642+0.0im
  0.383211-0.415402im   0.582927-0.995947im       0.233833+0.0im
   0.96662-0.532535im   0.767335-0.698466im       0.729057+0.0im
  0.135843-0.985474im   0.481892-0.480955im       0.677768+0.0im
 0.0471384-0.362081im   0.226423-0.639292im  …    0.338006+0.0im
   0.16316+0.0im       0.0305836+0.0im           -0.924134-0.316373im
  0.494637+0.0im        0.643694+0.0im           -0.712896-0.997439im
  0.883533+0.0im        0.512021+0.0im           -0.358076-0.452619im
  0.228847+0.0im        0.468642+0.0im          -0.0997231+0.0im

10×10 Matrix{ComplexF64}:
  0.455247+0.0im          …    0.228847+2.64698e-23im
  0.190538-0.4716im            0.468642-5.55112e-17im
  0.383211-0.415402im          0.233833+4.44089e-16im
   0.96662-0.532535im          0.729057+0.0im
  0.135843-0.985474im          0.677768-4.44089e-16im
 0.0471384-0.362081im     …    0.338006+4.71845e-16im
   0.16316-9.92617e-24im      -0.924134-0.316373im
  0.494637+0.0im              -0.712896-0.997439im
  0.883533-1.32349e-23im      -0.358076-0.452619im
  0.228847+2.64698e-23im     -0.0997231-0.0im

## Part 3

implementation in place of LDL `LD!(A)`

In [ ]:
function LD!(A)
    
end